In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import mean_squared_error,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

In [3]:
def history_train_test_timewise_split(data,column,history_start_date='2010-01-01',history_end_date='2050-12-31',train_start_date='2010-01-01',train_end_date='2050-12-31',test_start_date='2010-01-01',test_end_date='2050-12-31'): 
    import pandas as pd
    try:
        history_start_date=pd.to_datetime(history_start_date)
        history_end_date=pd.to_datetime(history_end_date)
        train_start_date=pd.to_datetime(train_start_date)
        train_end_date=pd.to_datetime(train_end_date)
        test_start_date=pd.to_datetime(test_start_date)
        test_end_date=pd.to_datetime(test_end_date)
    except:
        raise ValueError('Either format is incorrect or date is in correct.Dates are expected as string and in "yyyy-mm-dd" format')
        return
    if data[column].isnull().sum()>0:
        raise ValueError('Split column contains null values')
        return
    else:
        data[column]=pd.to_datetime(data[column])
        history=data.loc[(data[column]>=history_start_date)&(data[column]<=history_end_date)]
        train=data.loc[(data[column]>=train_start_date)&(data[column]<=train_end_date)]
        test=data.loc[(data[column]>=test_start_date)&(data[column]<=test_end_date)]
        left_history=data.loc[(data[column]>=history_start_date)&(data[column]<train_start_date)]
        history.reset_index(drop=True,inplace=True)
        train.reset_index(drop=True,inplace=True)
        test.reset_index(drop=True,inplace=True)
        return history,train,left_history,test

In [6]:
dataset = pd.read_csv("covid_19_india.csv",error_bad_lines=False)

In [7]:
dataset.shape

(15806, 9)

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [9]:
raw_dataset = dataset

In [10]:
dataset.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [11]:
dataset.isna().any()

Sno                         False
Date                        False
Time                        False
State/UnionTerritory        False
ConfirmedIndianNational     False
ConfirmedForeignNational    False
Cured                       False
Deaths                      False
Confirmed                   False
dtype: bool

In [12]:
no_use = ['Sno','Time']

In [13]:
dataset.drop(columns = no_use, inplace = True)
print("The shape of remaining data: ",dataset.shape)

The shape of remaining data:  (15806, 7)


In [14]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
dataset['State/UnionTerritory']= label_encoder.fit_transform(dataset['State/UnionTerritory'])
  
dataset['State/UnionTerritory'].unique()

array([19, 35, 11, 31, 38, 14, 20, 33, 18, 23, 30, 16,  1, 39, 28, 29, 40,
        7,  6, 13, 15, 22,  4, 24, 26,  0, 12, 37,  3, 17,  2, 36, 27, 25,
        9,  5, 32, 10, 21, 34,  8])

In [15]:
max(dataset['Date'])

'2021-06-08'

In [16]:
min(dataset['Date'])

'2020-01-30'

In [17]:
dispute_history,train,left_history,test=history_train_test_timewise_split(dataset.copy(),
                                                                 'Date',
                                                                 history_start_date='2020-01-30',
                                                                 history_end_date='2021-06-08',
                                                                 train_start_date='2020-01-30',
                                                                 train_end_date='2021-03-15',
                                                                 test_start_date='2021-03-16',
                                                                  #test_end_date='2019-05-12'
                                                                 )

In [18]:
print("Shape of dispute_history: ",dispute_history.shape)
print("Shape of train: ",train.shape)
print("Shape of left_history: ",left_history.shape)
print("Shape of test: ",test.shape)

Shape of dispute_history:  (15806, 7)
Shape of train:  (12746, 7)
Shape of left_history:  (0, 7)
Shape of test:  (3060, 7)


In [19]:
print("Percentage of data in train: ",((train.shape[0]/dispute_history.shape[0])*100))
print("Percentage of data in test: ",((test.shape[0]/dispute_history.shape[0])*100))

Percentage of data in train:  80.64026319119321
Percentage of data in test:  19.359736808806783


In [20]:
# Date features
# date features
def date_features(train, dispute_history, test, date_col):
    train['weekday'] = train[date_col].dt.weekday
    
    dispute_history['weekday'] = dispute_history[date_col].dt.weekday
    
    test['weekday'] = test[date_col].dt.weekday
    
    return train, dispute_history, test

# forming the date features
train, dispute_history, test = date_features(train, dispute_history, test, 'Date')

In [21]:
train.head()

,Date,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,weekday
0,2020-01-30,19,1,0,0,0,1,3
1,2020-01-31,19,1,0,0,0,1,4
2,2020-02-01,19,2,0,0,0,2,5
3,2020-02-02,19,3,0,0,0,3,6
4,2020-02-03,19,3,0,0,0,3,0


In [22]:
res = train['Confirmed']

In [23]:
res_test = test['Confirmed']

In [24]:
test.head()

,Date,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,weekday
0,2021-03-16,1,-,-,883380,7185,892008,1
1,2021-03-16,0,-,-,4966,62,5031,1
2,2021-03-16,2,-,-,16783,56,16840,1
3,2021-03-16,3,-,-,215106,1099,217817,1
4,2021-03-16,4,-,-,261171,1552,263051,1


In [25]:
train.drop(columns = ['Date','ConfirmedIndianNational','ConfirmedForeignNational','Confirmed'], inplace = True)

In [26]:
test.drop(columns = ['Date','ConfirmedIndianNational','ConfirmedForeignNational','Confirmed'], inplace = True)

In [27]:
print("The shape of remaining data: ",train.shape)

The shape of remaining data:  (12746, 4)


In [28]:
from sklearn import linear_model
reg_linear = linear_model.LinearRegression()

In [29]:
# fit the model to start training.
reg_linear.fit(train, res)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
# on test set
pred_test = reg_linear.predict(test)

In [31]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(res_test, pred_test, squared=False)
rms

98977.34741719451

In [32]:
rms/test.shape[0]

32.345538371632195

In [33]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators= 500 ,max_depth=3, random_state=0)
regr.fit(train, res)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=3, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [34]:
pred_test = regr.predict(test)

In [32]:
rms = mean_squared_error(res_test, pred_test, squared=False)
rms

505675.7461059856

In [33]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(train,res)

DecisionTreeRegressor(random_state=0)

In [34]:
pred_test = regressor.predict(test)

In [35]:
rms = mean_squared_error(res_test, pred_test, squared=False)
rms

505675.7461059856

In [36]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

In [37]:
regr.fit(train, res)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [38]:
pred_test = regr.predict(test)

In [39]:
rms = mean_squared_error(res_test, pred_test, squared=False)
rms

979634.2677058203

In [40]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=1)

clf.fit(train,res)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [41]:
pred_test = clf.predict(test)

In [42]:
rms = mean_squared_error(res_test, pred_test, squared=False)
rms

98977.33985271731

In [43]:
rms/test.shape[0]

32.345535899580824

In [44]:
r2_score = clf.score(test,res_test)
print(r2_score*100,'%')

98.60289369667173 %
